(data_processing)=
# Data processing

In [1]:
%load_ext watermark
# %load_ext rpy2.ipython
import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sns
# import rpy2
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import beta
from scipy.stats import multinomial


import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from myst_nb import glue
from slugify import slugify

from typing import List



# from rpy2.robjects.packages import importr

import plastockconf as psc
import plastock as pstk

from plastockconf import name_zones, name_frequentation, name_situation
from plastockconf import name_substrate, name_distance, table_css_styles, table_css_styles_top

from plastock import add_table_to_page, capitalize_x_tick_labels, capitalize_x_and_y_axis_labels, capitalize_legend_components, attribute_summary

import reportclass as rc

section = "A"
page = "5"

label = f'Table {section}{page}-'

def make_exportable(data, file_name, cmap='YlOrBr'):
    data.fillna(0, inplace=True)
    fig, ax = plt.subplots(figsize=(12,8))
    sns.heatmap(data=data, vmin=0, vmax=1, cmap=cmap, annot=True, fmt='.2', annot_kws={'size':10}, ax=ax, cbar=False)
    plt.tight_layout()
    ax.tick_params(which='both', axis='both', bottom=False, left=False)
    plt.savefig(file_name, dpi=300)

    plt.close()

glue('blank_caption', " ", display=False)

## Description des données macroplastiques

__Cette méthode s'applique aux données sur les déchets visibles à l'oeil nu, mais pas aux microplastiques__

### Position

Les inventaires des déchets menés précédemment sur le lac considéraient la plage comme une unité à part entière. Dans Pla'stock, chaque plage a été divisée en deux sections, la ligne d'eau et la plage sèche. Pour comparer les données de Pla'stock aux résultats précédents et à d'autres études de ce type, les résultats de Pla'stock doivent être transformés en une section pour une plage. En d'autres termes, nous devons combiner la surface des deux sections pour chaque échantillon.

### Objets

La variété des objets identifiés lors de chaque action dépend des déchets sur le terrain et du temps passé à compter, trier et identifier chaque objet collecté. Par conséquent, les seuls codes ou objets pris en considération sont ceux qui ont été identifiés pendant Pla'stock. Les données historiques sont filtrées pour les mêmes codes.

#### Combinaison d'objets similaires

Certaines catégories d'objets sont combinées. Cela signifie que nous modifions le code de certains éléments pour refléter les tendances de l'échantillonnage et tenir compte des différentes interprétations du protocole. Le tableau suivant résume la façon dont les codes sont combinés :

1. Gfrags: g80, g79, g78, g77, g76, g75
2. Gfoams: G81, G82, G76
3. Gcaps: G21, G22, G23, G24

Gfrags comprend toutes les formes de plastiques fragmentés de plus de 5 mm, Gfoams toutes les tailles de polystyrène expansé et Gcaps tous les bouchons en plastique et les anneaux en plastique qui accompagnent chaque bouchon.

In [2]:
# ! read in data !
# survey data
new_data = pd.read_csv("data/end_pipe/macro_current.csv")

# the position, substrate, time, area, length for each sample in the data
# ! there are samples that have different substrates on the same sample_id.!
# that is position 1 has a different substrate than position 2. for litter 
# data and comparing to previous results this is unimportant. The value to
# compare is the total survey divided by the length or the area. This needs
# to be accounted for and changes made to certain samples.
beach_data = pd.read_csv("data/end_pipe/pstock_beaches_current.csv")


# the feature columns of the survey data that are not dependent on the
# position or substrat variables. they are indexed on the Plage column
# one row for each sample location
beach_datax = pd.read_csv("data/end_pipe/asl_beaches.csv").set_index('Plage')

# The code definitions
codes = pd.read_csv('data/end_pipe/codes.csv').set_index('code')

# the regional labels for each survey location
regions = pd.read_csv("data/end_pipe/lac_leman_regions.csv")
regions.set_index('slug', drop=True, inplace=True)

# the city name of the survey locations
city_map = pd.read_csv('data/end_pipe/city_map.csv')
city_map.set_index('slug', inplace=True)

# translation of common terms into french, german and english
language_maps = rc.language_maps()

new_column_names = {
    "Position":"position",
    "Substrat":"substrat",
    "Date":"date",
    "Code":"code",
    "Quantité":"quantité",
    "Aire":"area"
}
new_data.loc[new_data.Substrat.isna(), "Substrat"] = 1

# import data and assign new column names and sample_id
# the sample_id is the tuple (location, date). Each row
# is a unique combinantion of sample_id and code
work_data = new_data[["Plage", *new_column_names.keys()]].copy()
work_data.rename(columns=new_column_names, inplace=True)
work_data["slug"] = work_data.Plage.apply(lambda x: slugify(x))
work_data["échantillon"] = list(zip(work_data.slug, work_data['date']))
work_data['date'] = pd.to_datetime(work_data["date"], format="mixed", dayfirst=True)
# work_data.dropna(inplace=True)



In [3]:
# type the columns
work_data[["position", "substrat"]] = work_data[["position", "substrat"]].astype("int")
work_data['échantillon'] = work_data['échantillon'].apply(lambda x: str(x))

In [5]:
voi = "substrat"
vals = "pcs/m²"
code = "G27"
loc_dates_t = ["('clarens', '16.01.2022')", "('amphion', '01.02.2022')"]

gfrags_c = codes.loc[codes.parent_code == 'Gfrags'].index

start_amph_s = new_data[(new_data.Plage == "Amphion")&(new_data["Date"] == "01.02.2022")&(new_data.Code == "G27")]
start_amph_c = new_data[(new_data.Plage == "Amphion")&(new_data["Date"] == "01.02.2022")&(new_data.Code.isin(gfrags_c))]
start_clarens_s = new_data[(new_data.Plage == "Clarens")&(new_data["Date"] == '16.01.2022')&(new_data.Code == "G27")]
start_clarens_c = new_data[(new_data.Plage == "Clarens")&(new_data["Date"] == '16.01.2022')&(new_data.Code.isin(gfrags_c))]

In [6]:
start_data = pd.concat([start_amph_s, start_amph_c, start_clarens_c, start_clarens_s])
start_data['pcs/m²'] = start_data["Quantité"]/start_data["Aire"]
# start_data.loc[start_data.Code.isin(gfrags_c), 'Code'] = 'Gfrags'
sd = start_data[["Plage", "Aire", "Position", "Substrat", "Date", "Code", "Quantité", 'pcs/m²']].copy()
sd.reset_index(inplace=True, drop=True)

display_columns = ["Plage", "Date", "Position", "Substrat", "Aire",  "Code", "Quantité", "pcs/m²"]

caption = f"<b>{label}1 :</b> Les résultats de l'échantillon avant la combinaison des codes, des substrats et de la position"
sd = sd[display_columns].style.set_table_styles(table_css_styles).format(**psc.format_kwargs).set_caption(caption).hide(axis="index")

glue("start_data", sd, display=False)

In [8]:
# ! locate all the duplicate values by sample id and area !
# this gives a data frame that has the position and area for
# each sample_id
total_area_dup = work_data.drop_duplicates(['échantillon', 'area'])

# ! combine the surface areas of the position vectors !
# sum of the areas for each position at each sample
# use the sample_id as index and sum the areas for each postition at each sample
total_area = total_area_dup.groupby(['échantillon', 'Plage'], as_index=False).area.sum()
total_area.set_index("échantillon", inplace=True)

# sum of the areas for each position at each sample
# use the sample_id as index and sum the areas for each postition at each sample
#total_area = total_area[['échantillon', 'area']].set_index('échantillon', drop=True)

In [13]:
# apply the total area to the work_data, index on sample_id
work_data['area_c'] = work_data['échantillon'].apply(lambda x: total_area.loc[x, "area"])
work_data['area'] = work_data.area_c
work_data.drop('area_c', axis=1, inplace=True)
# the code total per sample with the combined area
work_data = work_data.groupby(['échantillon', 'Plage', 'substrat', 'date', 'area','slug', 'quantité', 'code'], as_index=False)['quantité'].sum()
work_data.reset_index(inplace=True, drop=True)

In [14]:
mask = (work_data["échantillon"].isin(loc_dates_t))&(work_data.code.isin([*gfrags_c, code]))

display_columns = ["échantillon", "Plage", "Substrat", "Aire",  "Code", "Quantité"]
d = "pcs/m²"

test = work_data[mask].copy()
test = test.rename(columns={"substrat":"Substrat", "area":"Aire", "date":"Date", "code":"Code", "quantité":"Quantité"})
test = test[display_columns].copy()
# test.reset_index(inplace=True, drop=True)
caption = f"<b>{label}2 :</b> La variable position est supprimée et les surfaces sont combinées pour chaque position de chaque échantillon. Cependant, le substrat à Clarens n'est pas corrigé."
sd_2 = test.style.set_table_styles(table_css_styles).format(**psc.format_kwargs).set_caption(caption).hide(axis='index')
glue("start_data_2", sd_2, display=False)


In [15]:
# the code total per sample with the combined area
work_data = work_data.groupby(['échantillon', 'Plage', 'substrat', 'date', 'area','slug', 'code'], as_index=False)['quantité'].sum()

# add the regional component
work_data['region'] = work_data.slug.apply(lambda x: regions.loc[x, 'alabel'])

work_data = work_data.groupby(['échantillon', 'Plage', 'substrat', 'date', 'area','slug', 'quantité', 'code'], as_index=False)['quantité'].sum()
# get the pcs/m²  for each object at each sample
work_data['pcs/m²'] = work_data['quantité']/work_data.area

In [16]:
# ! there are samples that have different substrates on the same sample_id.!
# there should be one substrate per sample_id. Identify the locations that have duplicate values
some_data = work_data.copy()
groupby = ['échantillon', voi]
data = some_data.groupby(groupby, as_index=False)[vals].sum()

# the samples with more than one substrate
dd = data[data['échantillon'].duplicated()].copy()
# select all the duplicated sample_ids from the work_data
duplicated = work_data[work_data['échantillon'].isin(dd['échantillon'].unique())].copy()
# select all the duplicated sample_ids from the work_data
# change the substrat to [2]
duplicated['substrat'] = 2 

# select all the values that are not duplicated
not_duplicated = work_data[~(work_data['échantillon'].isin(dd['échantillon'].unique()))].copy()

# put it back together again
work_data = pd.concat([duplicated, not_duplicated])

In [17]:
mask = (work_data["échantillon"].isin(loc_dates_t))&(work_data.code.isin([*gfrags_c, code]))

display_columns = ["échantillon", "Plage", "Substrat", "Aire",  "Code", "Quantité", "pcs/m²"]

test = work_data[mask].copy()
test = test.rename(columns={"substrat":"Substrat", "area":"Aire", "date":"Date", "code":"Code", "quantité":"Quantité"})
test = test[display_columns].copy()

caption = f"<b>{label}3 :</b> Après avoir combiné les codes et supprimé la variable de la position, il reste des codes en double pour l'échantillon de Clarens."
sd_3 = test.style.set_table_styles(table_css_styles).format(**psc.format_kwargs).set_caption(caption).hide(axis="index")
glue("start_data_3", sd_3, display=False)

In [18]:
# ! valid codes and definitions !
# plastock did not use the same inventory as iqaasl
# here we select only the codes in the plastock inventory
pcodes = work_data.code.unique()

# identify and remove codes for which there is no defintion
# if the code is not defined then it can not be used
t = [x for x in pcodes if x not in codes.index]
wd_ni = work_data[~work_data.code.isin(t)].copy()

# ! aggregating to Gfrags, Gcaps and Gfoams !
# these items are not well divided into the composite subgroups
# for example people often know what a cap is, but whether it 
# comes from a drink bottle or other type is not well considered
# we combine the subcategories into more comprehensive groups.
ti = rc.use_gfrags_gfoams_gcaps(wd_ni, codes)

In [19]:
# ! groupby the sample id and code otherwise there are duplicate codes
# after aggregating to Gfrags etc..
work_data = ti.groupby(['échantillon', 'Plage', 'substrat', 'date', 'area', 'slug', 'code'], as_index=False).agg({'quantité':'sum'})
work_data['pcs/m²'] = work_data['quantité']/work_data['area']

# accounting for objects not found at a sample:
# the codes that were indentified are the unique codes
# in the set of data, they are the 'inventory'
codes_ip = work_data.code.unique()
# the unique samples by id
loc_dates = work_data['échantillon'].unique()

# a copy for itterating
wd = work_data.copy()

In [20]:
start_data.loc[start_data.Code.isin(gfrags_c), 'Code'] = 'Gfrags'

display_columns = ["Plage", "Date", "Position", "Substrat", "Aire",  "Code", "Quantité", "pcs/m²"]

start_data = start_data.groupby(["Plage", "Date", "Aire", "Code"], as_index=False)["Quantité"].sum()

start_data["pcs/m²"] = start_data["Quantité"]/start_data.Aire
sd_x = start_data.groupby(["Plage", "Date", "Code"], as_index=False)["pcs/m²"].mean()

caption =  f"<b>{label}5 :</b> La densité de l'échantillon si les zones et les substrats ne sont pas combinés pour chaque échantillon." 
sd_x = sd_x.style.set_table_styles(table_css_styles).format(**psc.format_kwargs).set_caption(caption).hide(axis="index")
glue("start_data_4", sd_x, display=False)

In [21]:
# a copy for itterating
wd = work_data.copy()

rows = []
for a_loc in loc_dates:
    r = wd.loc[wd['échantillon'] == a_loc].copy()
    r.reset_index(inplace=True, drop=True)
    
    t = r.loc[0][['échantillon', 'Plage', 'substrat', 'date', 'area', 'slug']].values
    asamp = [x for x in t]
    used_codes = r.code.unique()
    unused = [x for x in codes_ip if x not in used_codes]
    for element in unused:
        arow = [*asamp, element, 0, 0]
        rows.append(arow)
        

# now all the data has the same number of records per sample
# for each sample we can now say what was found and what was
# not found with respect to all the results
work_x = pd.DataFrame(rows, columns=['échantillon', 'Plage', 'substrat', 'date', 'area', 'slug', 'code', 'quantité', 'pcs/m²'])
work_data = pd.concat([work_x, work_data])


# add the regional component
work_data['region'] = work_data.slug.apply(lambda x: regions.loc[x, 'alabel'])

In [22]:
mask = (work_data["échantillon"].isin(loc_dates_t))&(work_data.code.isin(["Gfrags", "G27"]))
display_columns = ["échantillon", "Plage", "Substrat", "Aire",  "Code", "Quantité", "pcs/m²"]
test = work_data[mask].copy()
test = test.rename(columns={"substrat":"Substrat", "area":"Aire", "date":"Date", "code":"Code", "quantité":"Quantité"})
test = test[display_columns].copy()

caption =  f"<b>{label}4 :</b> The density of sample if the areas are combined. The result for each sample is the sum of the different results for the different positions and substrates for each sample."
sd_5 = test.style.set_table_styles(table_css_styles).format(**psc.format_kwargs).set_caption(caption).hide(axis="index")
glue("start_data_5", sd_5, display=False)

In [23]:
work_data.to_csv('data/end_pipe/macro_data_msquared.csv', index=False)
work_data_example = work_data.copy()

### Exemple

#### Mètres carrés

Pour illustrer les changements apportés aux données, considérons deux échantillons et deux codes (G27, Gfrags). Le premier échantillon est Amphion le 2022-02-01 et le second est Clarens le 2022-01-16. À Amphion, il y a plusieurs valeurs pour les mêmes codes le même jour. Il en va de même pour Clarens, mais à Clarens, il y a également deux substrats pour la plage. Notez que les codes G78 et G79 font partie du code combiné Gfrags.

```{glue} start_data
```

Pour combiner les données, nous devons additionner les surfaces pour chaque échantillon, appliquer la nouvelle surface à tous les comptages d'objets pour cet échantillon et ce jour, et supprimer la variable de position. Pour Amphion, la surface totale est de 342 + 98 = 440 m², pour Clarens 273 + 67 = 340 m².

```{glue} start_data_2
``` 

Après avoir combiné tous les totaux de codes pour chaque échantillon et remplacé le substrat 4 par le substrat 2 dans toutes les localités qui ont des valeurs de substrat distinctes pour la position, nous nous retrouvons encore avec des codes en double sur un échantillon.

```{glue} start_data_3
``` 

Dans la dernière étape, les codes qui doivent être combinés (G78 et G79) sont placés sous un seul code et les données sont agrégées à l'identifiant de l'échantillon (date et lieu de l'échantillon).  

```{glue} start_data_5
```

#### La différence

Si les surfaces des positions ne sont pas combinées, les valeurs attendues sont beaucoup plus faibles. En effet, nous sommes obligés de prendre la moyenne de la densité des deux positions et des substrats pour chaque échantillon. 

```{glue} start_data_4
``` 

__Problème de communication :__ Cela peut être difficile à communiquer. La moyenne pour une région est la moyenne de la densité totale par échantillon. C'est-à-dire le nombre total d'objets divisé par la surface totale. Cependant, dans ce cas, la moyenne de la région est la moyenne des moyennes par échantillon.

Cela signifie que la médiane d'une région serait la médiane de la moyenne des moyennes par échantillon. 


#### Mètres linéaires

L'opération de transformation des données en mètres linéaires est différente. Pour convertir toutes les plages en une seule section, nous abandonnons la variable superficie et la remplaçons par les données de longueur fournies. La longueur est la même pour les deux sections de chaque plage (les surfaces sont différentes), nous avons donc divisé la somme de tous les objets par la longueur donnée pour l'échantillon. Ce processus est conforme à la norme décrite dans le guide de surveillance des déchets marins.

In [24]:
# reading in data
new_datai = pd.read_csv("data/end_pipe/macro_current.csv")
beach_data = pd.read_csv("data/end_pipe/pstock_beaches_current.csv")


# adding the lenght of each sample to the results
length_key = beach_data[["Plage","length"]].drop_duplicates("Plage").set_index("Plage")
work_datai = new_datai[["Plage", *new_column_names.keys()]].copy()
work_datai.rename(columns=new_column_names, inplace=True)
work_datai["length"] = work_datai.Plage.apply(lambda x: length_key.loc[x, "length"])

# making a sample id from the location and date
work_datai["slug"] = work_datai.Plage.apply(lambda x: slugify(x))
work_datai["echantillon"] = list(zip(work_datai.slug, work_datai['date']))
work_datai['date'] = pd.to_datetime(work_datai["date"], format="mixed", dayfirst=True)
work_datai.dropna(inplace=True)

# defining the data type of columns
work_datai[["position", "substrat"]] = work_datai[["position", "substrat"]].astype("int")

# changing the column name to human readable
work_datai['échantillon'] = work_datai['echantillon']
work_datai.drop(['echantillon'], inplace=True, axis=1)

# removing the position variable from data and
# grouping by sample id and code to get the total per sample per code
work_datai = work_datai.groupby(['échantillon', 'Plage', 'substrat', 'date', 'length', 'slug', 'code'], as_index=False).agg({'quantité':'sum'})

# divide that by length
work_datai['pcs/m'] = work_datai['quantité']/work_datai['length']

# add the region
work_datai['region'] = work_datai.slug.apply(lambda x: regions.loc[x, 'alabel'])

In [25]:
# ! combining the substrates for beaches where there are two substrates !
voi = 'substrat'
vals = "pcs/m"
some_data = work_datai.copy()
groupby = ['échantillon', voi]
data = some_data.groupby(groupby, as_index=False)[vals].sum()

# these are the duplicate values that need to be changed
dd = data[data['échantillon'].duplicated()].copy()

duplicated = work_datai[work_datai['échantillon'].isin(dd['échantillon'].unique())].copy()
duplicated['substrat'] = 2 

# notduplicated
not_duplicated = work_datai[~(work_datai['échantillon'].isin(dd['échantillon'].unique()))].copy()

# put it back to gether again
work_datai = pd.concat([duplicated, not_duplicated])

# groupby the new substrat values and calculate pcs/m
work_datai = work_datai.groupby(['échantillon', 'Plage', 'region', 'substrat', 'date', 'length', 'slug', 'code'], as_index=False).agg({'quantité':'sum'})
work_datai['pcs/m'] = work_datai['quantité']/work_datai['length']

In [26]:
# ! defining the total inventory of the current data !
# accounting for objects not found at a sample:
# the codes that were indentified at least once = inventory
# defining the components of the inventory
codes_ip = work_datai.code.unique()

# the unique samples
loc_dates = work_datai['échantillon'].unique()

# a copy for itterating
wd = work_datai.copy()

# for each sample (échantillon) indentify the codes that were not
# found by indentifying all the codes that were found in all surveys
# and removing the codes that were not identified at that sample.
# for each unidentified code per sample, add a row with the sample
# id and the code. give the row a quantity of zero.
rows = []
for a_loc in loc_dates:
    r = wd.loc[wd['échantillon'] == a_loc].copy()
    r.reset_index(inplace=True, drop=True)
    
    t = r.loc[0][['échantillon', 'Plage', 'region', 'substrat', 'date', 'length', 'slug']].values
    asamp = [x for x in t]
    used_codes = r.code.unique()
    unused = [x for x in codes_ip if x not in used_codes]
    for element in unused:
        arow = [*asamp, element, 0, 0]
        rows.append(arow)
        

work_x = pd.DataFrame(rows, columns=['échantillon', 'Plage', 'region', 'substrat', 'date', 'length', 'slug', 'code', 'quantité', 'pcs/m'])
work_datai = pd.concat([work_x, work_datai])

work_datai.to_csv("data/end_pipe/macro_data_linearm.csv", index=False)

In [27]:
# micro plastics

# these samples were not completed

# drop_these = ['VD_Cul_2', 'VD_Vid_13', 'VD_Vid_8', 'VS_Bou_12']


# work_data = pd.read_csv("data/inprocess/micros_new_integrated.csv")
# work_data = work_data[~work_data["échantillon"].isin(drop_these)].copy()


# work_data.to_csv("data/end_pipe/long_form_micro.csv", index=False)

## Analyse et inférence

We used two methods of approximation to infer the expected results: _Random Forest Regression_ and _Bayesian grid approximation_. The random forest regression is centered around the average of the observed data. The samples drawn from the random forest regression do not approximate the distribution of the observed results, which are skewed. For predictions we have opted for the Bayesian grid approximation.

__Bayesian Grid Approximation:__
<br />

1. **Type of Model**: This approach is grounded in Bayesian statistics. It's not a specific model but a method for estimating or approximating the posterior distribution of model parameters.
2. **How it Works**: Involves specifying a prior distribution for the parameters, a likelihood function based on the data, and then using computational techniques (like grid approximation) to estimate the posterior distribution of the parameters.
3. **Key Features**:
   - **Uncertainty Estimation**: Provides a probabilistic interpretation and a way to estimate uncertainty in predictions.
   - **Prior Knowledge**: Incorporates prior knowledge or beliefs about the parameters through the prior distribution.
   - **Computationally Intensive**: For high-dimensional parameter spaces, grid approximation can become impractical.
   - **Flexibility**: Can be applied to a wide range of models, including linear models, hierarchical models, etc.

__Key Differences with random forest:__
<br />

- **Fundamental Approach**: Random Forest is a machine learning algorithm based on decision trees, while Bayesian Grid Approximation is a statistical method for estimating parameter distributions.
- **Output**: Random Forest provides a single predictive model. Bayesian methods provide a distribution of possible models, giving a sense of uncertainty.
- **Complexity and Computation**: Random Forest is generally straightforward and less computationally intensive compared to Bayesian methods, especially for large datasets or models with many parameters.
- **Interpretability**: Random Forests can be less interpretable due to the ensemble nature of many trees, whereas Bayesian methods offer probabilistic interpretations that can be insightful but might require a deeper statistical understanding.

Each method has its strengths and is suitable for different kinds of problems and data sets. The choice between them depends on the specific requirements of the analysis, such as the need for uncertainty quantification, computational resources, and the nature of the data.

In [28]:
# reading in data
# new_data = pd.read_csv("data/end_pipe/macro_current.csv")
work_data = pd.read_csv("data/end_pipe/macro_data_linearm.csv")
beach_data = pd.read_csv("data/end_pipe/pstock_beaches_current.csv")

# historical data
hist_leman = pd.read_csv("data/end_pipe/hist_leman.csv")
# ghi = pd.read_csv('data/end_pipe/iqaasl.csv')
hiqaasl = hist_leman.copy()
hiqaasl.rename(columns={"sample_id":"loc_date", "location":"slug", 'pcs/m':'pcs_m'}, inplace=True)
iq_cols = ['loc_date', 'date', 'slug', 'code', 'quantity', 'city', 'feature_name', 'feature_type', 'parent_boundary', 'pcs_m', 'project', 'region']

ghi = hiqaasl[iq_cols].copy()



# most recent results not plastock
ssp=pd.read_csv('data/end_pipe/swt_all.csv')

# # the independent variables are in asl_beaches file
beach_data_f = pd.read_csv("data/end_pipe/asl_beaches.csv").set_index('Plage')

# # code definitions
# codes = pd.read_csv('data/end_pipe/codes.csv').set_index('code')

# add the regional component
# the regional labels for each survey location
regions = pd.read_csv("data/end_pipe/lac_leman_regions.csv")
regions.set_index('slug', drop=True, inplace=True)

# # the city designation is used for reporting
# # the city name of the survey locations
# city_map = pd.read_csv('data/end_pipe/city_map.csv')
# city_map.set_index('slug', inplace=True)

codes_ip = work_data.code.unique()

change_names = ['preverenges', 'tolochenaz', 'versoix', 'vidy', 'cully']

plastock_cols = ['loc_date', 'date','slug','region', 'code', 'quantity', 'city', 'feature_name', 'feature_type','parent_boundary', 'pcs_m']
features = ['frequentation', 'situation', 'orientation', 'distance']

changeus = work_data[work_data.slug.isin(change_names)].copy()
donotchange = work_data[~work_data.slug.isin(change_names)].copy()

new_slug = {
    'cully': 'cully-p',
    'preverenges': 'preverenges-p',
    'tolochenaz': 'tolochenaz-p',
    'versoix':'versoix-p',
    'vidy': 'vidy-p'}

# they have the same name as locations in iqaasl
changeus['new_slug'] = changeus.slug.apply(lambda x: new_slug[x])
changeus['slug'] = changeus.new_slug
changeus.drop('new_slug', inplace=True, axis=1)

# the plastock data with the converted names
wd_nn = pd.concat([changeus, donotchange])

# # plastock did not use the same inventory as iqaasl
# # here we select only the codes in the plastock inventory
# pcodes = wd_nn.code.unique()

# identify and remove codes for which there is no defintion
# if the code is not defined then it can not be used
t = [x for x in codes_ip if x not in codes.index]
wd_ni = wd_nn[~wd_nn.code.isin(t)].copy()


# ! aggregating plastic caps, fragmented plastics, fragmented foams !
# these items are not well divided into the composite subgroups
# for example people often know what a cap is, but whether it 
# comes from a drink bottle or other type is not well considered
# we combine the subcategories into more comprehensive groups.
ti = rc.use_gfrags_gfoams_gcaps(wd_ni, codes)


# formatting data for reporting
# aggregate along all land-use and topo variables.
ti = ti.groupby(['échantillon', 'Plage', 'region', 'date', 'substrat', 'length', 'slug', 'code'], as_index=False).agg({'quantité':'sum'})

# !combinining with previous results!
# these are the default arguments for the report class
# the language maps gives the code definitions in english, german and french
# the top_label asserts the top level aggregation for the set of data defined by
# start, end dates and feature_name. These arguments are for the plastock data
language_maps = rc.language_maps()
top_label= ['feature_name', 'lac-leman']

# the default language is english in the report column class
# there are column names that need to be changed
new_names = {'échantillon': 'loc_date', 'pcs/m': 'pcs_m'}
ti.rename(columns={**new_names,'quantité': 'quantity'}, inplace=True)

# define the pcs/m column and the data to merge
ti['pcs_m'] = ti.quantity/ti.length

# adding and renaming columns according to reportclass requirements
# these values can be indexed on the IQAASL data
ti['city'] = ti.slug.apply(lambda x: city_map.loc[x])
ti['feature_name'] = 'lac-leman'
ti['feature_type'] = 'l'
ti['parent_boundary'] = 'rhone'

# ! adding feature columns to survey data !
# they can be merged on the Plage column and the index
# these are used for modeling
env_plastock = ti.merge(beach_data_f[features], left_on='Plage', right_index=True)
env_plastock = env_plastock[[*plastock_cols, *features, 'substrat']]

# ! the data for reporting !
ti_work = ti[plastock_cols].copy()

# this data is formatted to work with the reporting structure of IQAASL
# the landuse data is not included here.
ti_work = ti_work.groupby(plastock_cols, as_index=False).agg(psc.unit_agg)
ti_work['project']='Pla\'stock'

# merge the data and select only the current codes from plastock
txi = pd.concat([ghi, ti_work[[*plastock_cols, 'project']].copy()])
txi.reset_index(inplace=True)
txi = txi[txi.code.isin(ti_work.code.unique())]

# a report that includes both sets of data
boundaries = dict(start_date="2015-11-15", end_date="2023-01-01", feature_name="lac-leman", language="fr")
current = rc.ReportClass(txi.copy(), boundaries=boundaries, language="fr", lang_maps=language_maps, top_label=top_label)

In [29]:
mc, weight = current.most_common
mc.index

# they can be merged on the Plage column and the index
# env_plastock = ti.merge(beach_datax[features], left_on='Plage', right_index=True)

# ! creation of composite variables !
t_and_f = env_plastock.loc[:, ['loc_date', 'slug','date','code', 'pcs_m', 'quantity', 'frequentation', 'situation', 'distance', 'substrat', 'region']].copy()

# the substrat and distance features are being combined
# the two lowest and the two highest of each group are being combined
# substrat is a matter of combining different granularities. They are being grouped as
# sand and gravel.
# distance is now grouped by locations either less than or equal to 500 meters
t_and_f.loc[t_and_f.substrat <= 2, 'substrat'] = 1
t_and_f.loc[t_and_f.substrat > 2, 'substrat'] = 2
t_and_f.loc[t_and_f.distance <= 2, 'distance'] = 1
t_and_f.loc[t_and_f.distance > 2, 'distance'] = 2
t_and_f.loc[t_and_f.frequentation <= 2, 'frequentation'] = 2

# ! the data used in the models !
f_combi = t_and_f.copy()

f_combi.rename(columns={'frequentation':'fréquentation', 'loc_date': 'échantillon'}, inplace=True)


In [30]:
def sum_a_b(zipped):
    f = []
    for element in zipped:
        # # print(element[0])
        # # the new beta distribution would be
        # # total success, (total tries - total success)
        # obs = element[0]
        # obs2 = element[1]
        
        # new_element_0 = np.array([obs[0], obs[1] - obs[0]])
        # new_element_1 = np.array([obs2[0], obs2[1] - obs2[0]])
        t3 = element[0] + element[1]
        if t3[0] < 1 :
            t3 = np.array([1, 250])
        if t3[1] < 1:
            t3 = np.array([t3[0], 1])
        
        f.append(t3)
    return f
def draw_a_beta_value(posteriors):
    # d = next(generator)
    # drawing a random number from the beta distribution
    # this is the the chance p, that a binomial distribution will
    # result in True.
    my_beta = [beta.rvs(x[0], x[1], size=1) for x in posteriors]
    return my_beta


def calculate_likelihood(*, aggregated_data: pd.DataFrame, bin_density_column: str, pcs_column: str = 'pcs/m',
                         grid_range: np.ndarray = None, bins: list = None) -> pd.DataFrame:
    """
    Calculates the likelihood of observing the aggregated pcs/m data for each grid point and bin density value.

    Args:
        aggregated_data (pd.DataFrame): The aggregated data to be used for likelihood calculation.
        bin_density_column (str): The column representing bin density numbers.
        pcs_column (str, optional): The pcs/m column to use for calculation. Defaults to 'pcs/m'.
        grid_range (np.ndarray, optional): The range of grid values. Defaults to np.linspace(0, 9.99, 1000).

    Returns:
        pd.DataFrame: A DataFrame with likelihood values for each grid value and bin density number.
    """
    likelihood_df = pd.DataFrame(index=grid_range)
    likelihoods = []
    for bin_value in bins:
        bin_data = aggregated_data[aggregated_data[bin_density_column] == bin_value]
        
        if bin_data.empty:
            likelihoods = [np.array([1, 1]) for grid_point in grid_range]
        else:
            for grid_point in grid_range:
                passed = (bin_data[pcs_column] > grid_point).sum()
                tries = len(bin_data)
                fails = tries - passed
                if passed < 1:
                    fails=250
                likelihood = np.array([passed, fails])
                likelihoods.append(likelihood)
    likelihood_df[f'Likelihood_{bin_value}'] = likelihoods
    return likelihood_df


def binomial_probability_of_failure(generator):
    # in this case failure means exceeding the value
    # for trash a success is never exceeding the value
    d = next(generator)
    di = [x[0] for x in d]
    yield di
def define_posterior(likelihood, prior, grid_val_index: np.array = None):
    # the alpha, beta parameters of the likelihood and prior are assembled
    alpha_beta = list(zip(likelihood, prior))
    # this is a generator that yields the sum of the alpha, beta parameters
    # of the likelihood and prior. It generates one value for each point on the grid.
    a_b_sum = sum_a_b(alpha_beta)
    
    posteriors = [beta(x[0], x[1]).mean() for x in a_b_sum]
    for i in grid_val_index:
        # the sum of successes and failures for the scenario at the given
        # grid value are used as the alpha, beta parameters of the beta distribtion
        # for the binomial/bernouli probability that a sample will exceed the grid
        # value i.
        st = binomial_probability_of_failure(draw_a_beta_value(a_b_sum))
        val = next(st)
        posteriors.append(val)
    
    # return posterior probabilities with gird index and column labels
    post_grid_pstock = pd.DataFrame(posteriors, index=grid_val_index, columns=prior.columns)
    
    # identify the x scale of the grid
    post_grid_pstock['X'] = post_grid_pstock.index
    
    # this column is the normalized probabilities that a sample
    # will exceed a value on the grid.
    post_grid_pstock['norm'] = post_grid_pstock['Bin_1'] / post_grid_pstock['Bin_1'].sum()
    
    return post_grid_pstock
order = ["Haut lac", "Grand lac",  "Petit lac"]

def region_and_code(df, code, regions=order):
    # returns the survey results for a region and a code
    ds = []
    for region in regions:
        mask = (df.code == code)&(df.region == region)
        d = df[mask]
        d = d.groupby(['loc_date', 'project', 'region'], as_index=False).pcs_m.sum()
        d["region"] = ordinal[region]
        ds.append(d)
    return ds

def region_samp_total(df, regions=order):
    ds = []
    for region in regions:
        mask = (df.region == region)
        d = df[mask]
        d = d.groupby(['loc_date', 'project', 'region'], as_index=False).pcs_m.sum()
        d["region"] = ordinal[region]
        ds.append(d)
    return ds
    

In [31]:
prior_data = current.w_df.copy()
prior_data = prior_data[~prior_data.isna()]

prior_data['region'] = prior_data.slug.apply(lambda x: regions.loc[x, 'alabel'])
prior_data = prior_data.loc[prior_data.code.isin(mc.index)]
prd= prior_data[prior_data.project == "IQAASL"].copy()
ps= prior_data[prior_data.project == "Pla\'stock"].copy()


grid_val_index = np.linspace(0, 19.99, 2000)
groupby_columns = ['sample_id', 'location', 'date', 'city', 'orchards', 'vineyards', 'buildings', 'forest',
                   'undefined', 'public_services', 'streets']

ordinal = {x:i+1 for i,x in enumerate(order)}

beta_prior = pstk.calculate_beta_prior(grid_range=grid_val_index, bin_density_numbers=[1])

code_results = {}
for acode in mc.index:
    u = region_and_code(prd.copy(), acode)
    pk = region_and_code(ps.copy(), acode)
    
    hlprior = u[0]
    hlc = pk[0]
    results = {}
    for i, n in enumerate(order):
        hlx = calculate_likelihood(aggregated_data=u[i].copy(), bin_density_column='region', pcs_column='pcs_m', grid_range=grid_val_index, bins=[i+1])
        hcx = calculate_likelihood(aggregated_data=pk[i].copy(), bin_density_column='region', pcs_column='pcs_m', grid_range=grid_val_index, bins=[i+1])
        hl_v = [x[0] for x in hlx.values]
        hc_x = [x[0] for x in hcx.values]
        b_v = [x[0] for x in beta_prior.values]
    
        # the prior distribution
        ab = list(zip(hl_v, b_v))
        summed = sum_a_b(ab)
    
        # the likelihoood
        summedl = sum_a_b(list(zip(hc_x, b_v)))
    
        # the posterior
        sall = sum_a_b(list(zip(hl_v, hc_x)))
    
        prior_mean =  [beta(x[0], x[1]).mean() for x in summed]
        likeli = [beta(x[0], x[1]).mean() for x in summedl]
    
        p_fail = [beta(x[0], x[1]).mean() for x in sall]
        p_normed = p_fail/np.sum(p_fail)
    
        res_df = pd.DataFrame(index=grid_val_index)
        res_df['alpha-beta-post'] = sall
        res_df['prior-mean'] = prior_mean
        res_df['observed-mean'] = likeli
        res_df["beta-post"] = p_fail
        res_df["beta-norm"] = p_normed
        res_df["bi-post"] = res_df["prior-mean"]*res_df["observed-mean"]
        res_df["bi-norm"] = res_df["bi-post"]/res_df["bi-post"].sum()
        res_df["X"] = res_df.index
        results.update({n:res_df})
    code_results.update({acode:results})

samples = []
for acode in mc.index:
    cr = code_results[acode]
    res = {}
    for place in order:
        rv = multinomial(1, cr[place]["bi-norm"].values)
        y = rv.rvs(500)
        indexes = []
        for i in range(0, len(y)):
            nip = np.nonzero(y[i])[0]
            indexes.extend(nip)
        new_bi = pd.DataFrame(grid_val_index, columns=["X"])
        samps = new_bi.loc[indexes, "X"]
        res.update({place:samps})
    samples.append(res)


In [32]:
pst = region_samp_total(prd.copy(), regions=order)
cst = region_samp_total(ps.copy(), regions=order)

resultst = {}
for i, n in enumerate(order):
    hlx = calculate_likelihood(aggregated_data=pst[i].copy(), bin_density_column='region', pcs_column='pcs_m', grid_range=grid_val_index, bins=[i+1])
    hcx = calculate_likelihood(aggregated_data=cst[i].copy(), bin_density_column='region', pcs_column='pcs_m', grid_range=grid_val_index, bins=[i+1])
    hl_v = [x[0] for x in hlx.values]
    hc_x = [x[0] for x in hcx.values]
    b_v = [x[0] for x in beta_prior.values]

    # the prior distribution
    ab = list(zip(hl_v, b_v))
    summed = sum_a_b(ab)

    # the likelihoood
    summedl = sum_a_b(list(zip(hc_x, b_v)))

    # the posterior
    sall = sum_a_b(list(zip(hl_v, hc_x)))

    prior_mean =  [beta(x[0], x[1]).mean() for x in summed]
    likeli = [beta(x[0], x[1]).mean() for x in summedl]

    p_fail = [beta(x[0], x[1]).mean() for x in sall]
    p_normed = p_fail/np.sum(p_fail)

    res_df = pd.DataFrame(index=grid_val_index)
    res_df['alpha-beta-post'] = sall
    res_df['prior-mean'] = prior_mean
    res_df['observed-mean'] = likeli
    res_df["beta-post"] = p_fail
    res_df["beta-norm"] = p_normed
    res_df["bi-post"] = res_df["prior-mean"]*res_df["observed-mean"]
    res_df["bi-norm"] = res_df["bi-post"]/res_df["bi-post"].sum()
    res_df["X"] = res_df.index
    resultst.update({n:res_df})
samplest = []
for place in order:
    rv = multinomial(1, resultst[place]["bi-norm"].values)
    y = rv.rvs(500)
    indexes = []
    for i in range(0, len(y)):
        nip = np.nonzero(y[i])[0]
        indexes.extend(nip)
    new_bi = pd.DataFrame(grid_val_index, columns=["X"])
    samps = new_bi.loc[indexes, "X"]
    res.update({place:samps})
    samplest.append(res)

### Haut lac

In [33]:
display_language = current.language
display_language_map = current.lang_maps[display_language]
columns = ['mean', 'std', 'min', '25%', '50%', '75%', 'max']
hl_stot = pd.DataFrame(samplest[0]["Haut lac"].describe()).T
hl_stot  = hl_stot[columns]

hl_stot.index = ["Haut lac"]
hl_stot = rc.translated_and_style_for_display(hl_stot, display_language_map, display_language, gradient=False)

caption =  f"<b>{label}6 :</b> Haut lac, la distribution attendue des résultats des échantillons pour 2024." 
glue("start_data_6", hl_stot.set_caption(caption), display=True)

,moyenne,écart-type,min,25%,50%,75%,max
Haut lac,"3,17","3,16","0,00","0,82","2,26","4,45","16,40"


In [34]:
columns = ['mean', 'std', 'min', '25%', '50%', '75%', 'max', 'objet']
predictions = []
for i, n in enumerate(mc.index):
    xi = samples[i]["Haut lac"].describe()
    xi.loc["objet"] = n
    predictions.append(pd.DataFrame(xi).T)
hl_predictions = pd.concat(predictions)
hl_predictions = hl_predictions[columns]
hl_predictions.set_index('objet', inplace=True, drop=True)
hl_predictions.index.name = None
haut_lac = rc.translated_and_style_for_display(hl_predictions, display_language_map, display_language, gradient=False)
caption =  f"<b>{label}7 :</b> Haut lac, La distribution attendue des objets les plus courants 2024.." 
glue("start_data_7", haut_lac.set_caption(caption), display=True)

,moyenne,écart-type,min,25%,50%,75%,max
"Fragments de plastique: g80, g79, g78, g77, g76, g75","1,23","1,49","0,00","0,19","0,78","1,65","8,00"
Mégots et filtres à cigarettes,"0,52","0,73","0,00","0,09","0,26","0,58","4,82"
"Fragments de polystyrène expansé: g81, g82, g83","0,58","1,01","0,00","0,12","0,27","0,67","11,76"
"Emballages de bonbons, de snacks","0,39","0,96","0,00","0,07","0,18","0,43","15,25"
Coton-tige,"0,25","0,73","0,00","0,06","0,14","0,30","15,52"
"Couvercles en plastique bouteille: g21, g22, g23, g24","0,25","0,82","0,00","0,04","0,09","0,25","14,18"
"Bâche, feuille plastique industrielle","1,54","3,34","0,00","0,02","0,05","0,70","18,91"
Pellets industriels (gpi),"0,49","2,20","0,00","0,04","0,13","0,27","19,98"
Fragments de plastique angulaires <5mm,"0,49","1,71","0,00","0,06","0,17","0,42","15,35"
Déchets de construction en plastique,"0,33","1,31","0,00","0,05","0,14","0,36","19,77"


### Grand lac

In [35]:
columns = ['mean', 'std', 'min', '25%', '50%', '75%', 'max']
gl_stot = pd.DataFrame(samplest[0]["Grand lac"].describe()).T
gl_stot  = gl_stot[columns]

gl_stot.index = ["Grand lac"]
gl_stot = rc.translated_and_style_for_display(gl_stot, display_language_map, display_language, gradient=False)
caption =  f"<b>{label}8 :</b> Grand lac, la distribution attendue des résultats des échantillons pour 2024." 
glue("start_data_8", gl_stot.set_caption(caption), display=True)

,moyenne,écart-type,min,25%,50%,75%,max
Grand lac,"2,20","2,25","0,00","0,71","1,59","2,95","19,95"


In [36]:
columns = ['mean', 'std', 'min', '25%', '50%', '75%', 'max', 'objet']
predictions = []
for i, n in enumerate(mc.index):
    xi = samples[i]["Grand lac"].describe()
    xi.loc["objet"] = n
    predictions.append(pd.DataFrame(xi).T)
hl_predictions = pd.concat(predictions)
hl_predictions = hl_predictions[columns]
hl_predictions.set_index('objet', inplace=True, drop=True)
hl_predictions.index.name = None
grand_lac = rc.translated_and_style_for_display(hl_predictions, display_language_map, display_language, gradient=False)
caption =  f"<b>{label}9 :</b> Grand lac, La distribution attendue des objets les plus courants 2024." 
glue("start_data_9", grand_lac.set_caption(caption), display=True)

,moyenne,écart-type,min,25%,50%,75%,max
"Fragments de plastique: g80, g79, g78, g77, g76, g75","1,07","1,69","0,00","0,19","0,51","1,27","15,93"
Mégots et filtres à cigarettes,"0,49","0,72","0,00","0,08","0,22","0,53","6,39"
"Fragments de polystyrène expansé: g81, g82, g83","0,37","0,99","0,00","0,05","0,15","0,34","14,51"
"Emballages de bonbons, de snacks","0,21","0,20","0,00","0,06","0,14","0,31","1,05"
Coton-tige,"0,16","0,17","0,00","0,04","0,10","0,22","0,76"
"Couvercles en plastique bouteille: g21, g22, g23, g24","0,27","1,34","0,00","0,02","0,08","0,18","19,16"
"Bâche, feuille plastique industrielle","0,35","1,63","0,00","0,04","0,11","0,24","19,73"
Pellets industriels (gpi),"0,78","1,09","0,00","0,12","0,42","1,02","8,92"
Fragments de plastique angulaires <5mm,"0,60","1,52","0,00","0,07","0,27","0,70","19,72"
Déchets de construction en plastique,"0,51","2,31","0,00","0,01","0,02","0,04","18,04"


### Petit lac

In [37]:
columns = ['mean', 'std', 'min', '25%', '50%', '75%', 'max']
pl_stot = pd.DataFrame(samplest[0]["Petit lac"].describe()).T
pl_stot  = pl_stot[columns]

pl_stot.index = ["Petit lac"]
pl_stot = rc.translated_and_style_for_display(pl_stot, display_language_map, display_language, gradient=False)
caption =  f"<b>{label}10 :</b> Petit lac, la distribution attendue des résultats des échantillons pour 2024." 
glue("start_data_10", pl_stot.set_caption(caption), display=True)

,moyenne,écart-type,min,25%,50%,75%,max
Petit lac,"1,02","1,27","0,00","0,26","0,66","1,27","12,74"


In [38]:
columns = ['mean', 'std', 'min', '25%', '50%', '75%', 'max', 'objet']
predictions = []
for i, n in enumerate(mc.index):
    xi = samples[i]["Petit lac"].describe()
    xi.loc["objet"] = n
    predictions.append(pd.DataFrame(xi).T)
hl_predictions = pd.concat(predictions)
hl_predictions = hl_predictions[columns]
hl_predictions.set_index('objet', inplace=True, drop=True)
hl_predictions.index.name = None
petit_lac = rc.translated_and_style_for_display(hl_predictions, display_language_map, display_language, gradient=False)
caption =  f"<b>{label}11 :</b> Petit lac, La distribution attendue des objets les plus courants 2024." 
glue("start_data_11", petit_lac.set_caption(caption), display=True)

,moyenne,écart-type,min,25%,50%,75%,max
"Fragments de plastique: g80, g79, g78, g77, g76, g75","0,36","0,41","0,00","0,09","0,23","0,49","2,80"
Mégots et filtres à cigarettes,"0,32","0,53","0,00","0,06","0,17","0,35","5,93"
"Fragments de polystyrène expansé: g81, g82, g83","0,25","1,55","0,00","0,01","0,02","0,05","18,63"
"Emballages de bonbons, de snacks","0,11","0,14","0,00","0,04","0,06","0,13","0,90"
Coton-tige,"0,23","1,44","0,00","0,02","0,05","0,13","19,51"
"Couvercles en plastique bouteille: g21, g22, g23, g24","0,22","1,32","0,00","0,01","0,04","0,08","19,03"
"Bâche, feuille plastique industrielle","3,68","5,59","0,00","0,22","0,72","5,56","19,75"
Pellets industriels (gpi),"0,41","1,25","0,00","0,04","0,12","0,37","18,30"
Fragments de plastique angulaires <5mm,"0,58","1,46","0,00","0,08","0,23","0,73","19,31"
Déchets de construction en plastique,"0,77","2,99","0,00","0,01","0,02","0,04","19,95"


In [39]:
%watermark --iversions -b -r

Git repo: https://github.com/hammerdirt-analyst/plastock.git

Git branch: main

seaborn   : 0.13.1
matplotlib: 3.8.2
numpy     : 1.26.3
pandas    : 2.0.3



<!-- ### Analyse factorielle de données mixtes

L'ACP n'est pas recommandée si l'on utilise des types de données mixtes. Nous suivrons ici la méthode recommandée. Nous poursuivrons ensuite avec une ACP traditionnelle. Dans ce cas particulier, l'augmentation de la dimensionnalité causée par les variables nominales peut ne pas être un problème.

_"L’introduction simultanée de variables quantitatives et qualitatives (donnéesdites mixtes) en tant qu’éléments actifs d’une même analyse factorielle est une problématique fréquente. La méthodologie usuelle consiste à transformer les variables quantitatives en qualitatives en découpant en classes leur intervalle de variation et à soumettre le tableau homogène ainsi obtenu à une analyse des correspondances multiples (ACM)"._ (_citation: Revue de statistique appliquée, tome 52, no 4 (2004), p. 93-111_)

Cette section est rédigée en R. Les données sont les mêmes que celles utilisées dans les sections précédentes. -->

<!-- ### AFDM 5 dimensions

#### Valeurs Eigen et pourcentage cumulé de la variance -->